######################

**시간대 분석**
######################

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset from Kaggle
file_path1 = '/content/drive/MyDrive/Data/SOL60_INDICATOR3'
data = pd.read_csv(file_path1)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
data.columns

In [ ]:
# 사용하지 않을 열 제외
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
# 시간 관련 열 변환 함수
def convert_time_features(data):
    # open_time 열이 datetime 형식이 아닌 경우 변환
    if not np.issubdtype(data['open_time'].dtype, np.datetime64):
        data['open_time'] = pd.to_datetime(data['open_time'])

    # time 열을 분 단위로 변환
    data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

    # 사용하지 않을 열 제외
    data = data.drop(columns=['open_time'])

    return data
# 시간 관련 열 변환
data = convert_time_features(data)
data.head()

In [ ]:
# 무한대 값을 NaN으로 대체
data.replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN 값을 평균으로 대체
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)

# numpy 배열을 다시 DataFrame으로 변환
data = pd.DataFrame(data_imputed, columns=data.columns)

data.head()

In [ ]:
# time 열을 기준으로 그룹화하여 필요한 계산 수행
results = data.groupby('time').agg(
    total_rows=('max_return_60min', 'size'),
    count_above_1_1=('max_return_60min', lambda x: (x >= 1.1).sum())
)

# 비율 계산
results['percentage_above_1_1'] = (results['count_above_1_1'] / results['total_rows']) * 100

# 결과 출력
results = results.reset_index()

In [ ]:
# 시각화
plt.figure(figsize=(10, 6))
plt.plot(results['time'], results['percentage_above_1_1'], marker='o', linestyle='-')
plt.title('Percentage of max_return_60min >= 1.1 over Time')
plt.xlabel('Time')
plt.ylabel('Percentage of max_return_60min >= 1.1')
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# time 값을 시-분 형식으로 변환하는 함수
def convert_minutes_to_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    return f"{int(hours):02}:{int(mins):02}"

# time 열을 기준으로 그룹화하여 필요한 계산 수행
results = data.groupby('time').agg(
    total_rows=('max_return_60min', 'size'),
    count_above_1_1=('max_return_60min', lambda x: (x >= 1.1).sum())
)

# 비율 계산
results['percentage_above_1_1'] = (results['count_above_1_1'] / results['total_rows']) * 100

# time 값을 시-분 형식으로 변환
results = results.reset_index()
results['time'] = results['time'].apply(convert_minutes_to_time)

In [ ]:
# time 열을 기준으로 그룹화하여 필요한 계산 수행
results = data.groupby('time').agg(
    total_rows=('max_return_60min', 'size'),
    count_above_1_1=('max_return_60min', lambda x: (x >= 1.1).sum())
)

# 비율 계산
results['percentage_above_1_1'] = (results['count_above_1_1'] / results['total_rows']) * 100

# 결과 출력
results = results.reset_index()

# 슬라이딩 윈도우의 크기 설정 (예: 60분)
window_size = 60

# 슬라이딩 윈도우 평균 계산
results['rolling_mean'] = results['percentage_above_1_1'].rolling(window=window_size // 10, min_periods=1).mean()

# 임계값 설정
threshold = 33  # 여기서 임계값을 설정하세요

# 임계값 이상의 구간 필터링
high_prob_zones = results[results['rolling_mean'] >= threshold]

# 연속된 구간의 시작과 끝을 식별
high_prob_zones['group'] = (high_prob_zones['time'].diff() > 10).cumsum()

# 각 그룹의 시작과 끝 시간 구하기
grouped_zones = high_prob_zones.groupby('group').agg(
    start_time=('time', 'first'),
    end_time=('time', 'last')
).reset_index()

# 시간축 이동
shift_point = 200
before_shift = results[results['time'] < shift_point].copy()
after_shift = results[results['time'] >= shift_point].copy()

before_shift['shifted_time'] = before_shift['time'] + 1440
after_shift['shifted_time'] = after_shift['time']

results_shifted = pd.concat([after_shift, before_shift]).reset_index(drop=True)

# 시각화
plt.figure(figsize=(12, 6))
plt.plot(results_shifted['shifted_time'], results_shifted['percentage_above_1_1'], marker='o', linestyle='-', label='Percentage over Time')
plt.plot(results_shifted['shifted_time'], results_shifted['rolling_mean'], linestyle='-', color='orange', label='Rolling Mean')

# 임계값 선 추가
plt.axhline(y=threshold, color='purple', linestyle='-', label=f'Threshold {threshold}%')

# 영역 색칠
plt.fill_between(results_shifted['shifted_time'], threshold, results_shifted['percentage_above_1_1'], where=(results_shifted['percentage_above_1_1'] >= threshold), interpolate=True, color='purple', alpha=0.3)

# 높은 확률 구간에서 시작과 끝 구간만 표시
for _, row in grouped_zones.iterrows():
    if row['start_time'] < shift_point:
        shifted_start_time = row['start_time'] + 1440
    else:
        shifted_start_time = row['start_time']
    if row['end_time'] < shift_point:
        shifted_end_time = row['end_time'] + 1440
    else:
        shifted_end_time = row['end_time']
    plt.axvline(x=shifted_start_time, color='red', linestyle='--', label='Start Time' if row['start_time'] == grouped_zones['start_time'].iloc[0] else "")
    plt.axvline(x=shifted_end_time, color='green', linestyle='--', label='End Time' if row['end_time'] == grouped_zones['end_time'].iloc[0] else "")

# 0 지점을 나타내는 세로선 추가
plt.axvline(x=shift_point, color='blue', linestyle=':', label='Midnight Transition')

plt.title('Percentage of max_return_60min >= 1.1 over Time')
plt.xlabel('Time (minutes)')
plt.ylabel('Percentage of max_return_60min >= 1.1')
plt.xticks(rotation=45)
plt.xlim(shift_point, 1440 + shift_point)
plt.xticks(np.arange(shift_point, 1440 + shift_point + 1, 100), labels=[str(x % 1440) for x in np.arange(shift_point, 1440 + shift_point + 1, 100)])
plt.xticks(list(plt.xticks()[0]) + [0], labels=list(plt.xticks()[1]) + ['0'])  # 0을 x축에 추가
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

# 분을 시-분 형식으로 변환하는 함수
def convert_minutes_to_time(minutes):
    hours = minutes // 60
    mins = minutes % 60
    return f"{int(hours):02}:{int(mins):02}"

# 한국 시간으로 변환하는 함수
def convert_to_kst(utc_minutes):
    kst_minutes = (utc_minutes + 540) % 1440  # 한국 시간은 UTC+9 (540분)
    return convert_minutes_to_time(kst_minutes)

# 높은 확률 구간 시작과 끝 값 출력
print(f"High probability zones above {threshold}% (start_time, end_time):")
for _, row in grouped_zones.iterrows():
    start_time_utc = convert_minutes_to_time(row['start_time'])
    end_time_utc = convert_minutes_to_time(row['end_time'])
    start_time_kst = convert_to_kst(row['start_time'])
    end_time_kst = convert_to_kst(row['end_time'])
    print(f"Start: {start_time_utc} (KST: {start_time_kst}), End: {end_time_utc} (KST: {end_time_kst})")

######################

**요일분석**
######################

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset from Kaggle
file_path1 = '/content/drive/MyDrive/Data/SOL60_INDICATOR3'
data = pd.read_csv(file_path1)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
data.columns

In [ ]:
# 사용하지 않을 열 제외
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
# 시간 관련 열 변환 함수
def convert_time_features(data):
    # open_time 열이 datetime 형식이 아닌 경우 변환
    if not np.issubdtype(data['open_time'].dtype, np.datetime64):
        data['open_time'] = pd.to_datetime(data['open_time'])

    # 요일 추출하여 data_day 변수에 저장
    data_day = data['open_time'].dt.day_name()

    # time 열을 분 단위로 변환
    data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

    # 사용하지 않을 열 제외
    data = data.drop(columns=['open_time'])

    # 무한대 값을 NaN으로 대체
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    # NaN 값을 평균으로 대체
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)

    # numpy 배열을 다시 DataFrame으로 변환
    data = pd.DataFrame(data_imputed, columns=data.columns)

    data['day'] = data_day

    return data
# 시간 관련 열 변환
data = convert_time_features(data)
data.head()

In [ ]:
# 그룹화하여 집계 계산
results = data.groupby('day').agg(
    total_rows=('max_return_60min', 'size'),
    count_above_1_1=('max_return_60min', lambda x: (x >= 1.1).sum())
)

# 비율 계산
results['percentage_above_1_1'] = (results['count_above_1_1'] / results['total_rows']) * 100

# 결과 출력
print(results)

In [ ]:
# 요일 순서 정렬
ordered_days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
results = results.reindex(ordered_days)

# 결과 출력
print(results)

# 시각화
fig, ax = plt.subplots(figsize=(10, 6))

# 막대 그래프 생성
results['percentage_above_1_1'].plot(kind='bar', ax=ax, color='skyblue', edgecolor='black')

# 그래프 제목 및 축 레이블 설정
ax.set_title('Percentage of max_return_60min >= 1.1 by Day of Week')
ax.set_xlabel('Day of Week')
ax.set_ylabel('Percentage (%)')
ax.set_xticklabels(ordered_days, rotation=45, ha='right')

# 값 레이블 추가
for i in ax.containers:
    ax.bar_label(i, fmt='%.1f%%')

# 그래프 출력
plt.tight_layout()
plt.show()

######################

**변동에 대한 최대 하락 구간의 분포**
######################

In [ ]:
pip install mplfinance

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import mplfinance as mpf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset from Kaggle
file_path1 = '/content/drive/MyDrive/Data/SOL60_INDICATOR3'
data = pd.read_csv(file_path1)

# Display the first few rows of the dataset to ensure it is loaded correctly
data.head()

In [ ]:
data.columns

In [ ]:
# 사용하지 않을 열 제외
data = data.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

In [ ]:
# 시간 관련 열 변환 함수
def convert_time_features(data):
    # open_time 열이 datetime 형식이 아닌 경우 변환
    if not np.issubdtype(data['open_time'].dtype, np.datetime64):
        data['open_time'] = pd.to_datetime(data['open_time'])

    # 요일 추출하여 data_day 변수에 저장
    data_day = data['open_time'].dt.day_name()

    # time 열을 분 단위로 변환
    data['time'] = data['open_time'].dt.hour * 60 + data['open_time'].dt.minute

    # 사용하지 않을 열 제외
    data = data.drop(columns=['open_time'])

    # 무한대 값을 NaN으로 대체
    data.replace([np.inf, -np.inf], np.nan, inplace=True)

    # NaN 값을 평균으로 대체
    imputer = SimpleImputer(strategy='mean')
    data_imputed = imputer.fit_transform(data)

    # numpy 배열을 다시 DataFrame으로 변환
    data = pd.DataFrame(data_imputed, columns=data.columns)

    data['day'] = data_day

    return data
# 시간 관련 열 변환
data = convert_time_features(data)

# 시간 변환 함수
def convert_time_to_hhmm(df, time_col='time'):
    # 분 단위를 시-분 형식으로 변환
    df['hhmm_time'] = pd.to_datetime(df[time_col], unit='m').dt.strftime('%H:%M')
    return df

# 시간 변환 함수 적용
data = convert_time_to_hhmm(data)

data.head()

In [ ]:
# max_return_60min이 1.1 이상인 데이터 필터링
filtered_data = data[data['max_return_60min'] >= 1.5]

# hhmm_time을 datetime 형식으로 변환
filtered_data['hhmm_time'] = pd.to_datetime(filtered_data['hhmm_time'], format='%H:%M')

# hhmm_time 열을 기준으로 그룹화하여 min_return_60min의 분포를 캔들 스틱으로 표시
candlestick_data = filtered_data.groupby(filtered_data['hhmm_time']).agg(
    Open=('min_return_60min', 'first'),
    High=('min_return_60min', 'max'),
    Low=('min_return_60min', 'min'),
    Close=('min_return_60min', 'last')
)

# 인덱스를 DatetimeIndex로 설정
candlestick_data.index = pd.to_datetime(candlestick_data.index)

# 캔들 스틱 차트 시각화
mpf.plot(candlestick_data, type='candle', style='charles', title='Candlestick chart of min_return_60min', ylabel='min_return_60min')

plt.show()

In [ ]:
# max_return_60min과 min_return_60min 열을 마지막 부분으로 이동
columns = list(data.columns)
columns.remove('max_return_60min')
columns.remove('min_return_60min')
columns.extend(['max_return_60min', 'min_return_60min'])
data = data[columns]

In [ ]:
# max_return_60min이 1.1 이상이면서 min_return_60min이 1 이하인 데이터 필터링
filtered_data2 = data[(data['max_return_60min'] >= 1.5) & (data['min_return_60min'] <= -10)]

# 결과 출력
filtered_data2